深度学习的第一目的就是要在训练集上得到好的结果，然后用这个模型去测试集上验证，也就是第二目的，是过拟合，还是欠拟合。

模型并不总是会过拟合，也有可能是根本就没有train好，而产生的欠拟合。

对于如何在训练集上得到好的结果？

1. 优化梯度方向
2. 调整学习率
3. 选择合适的激活函数
4. BatchNormalization

而对于如何在验证集上得到好的结果呢？

1. 早停
2. 正则化
3. Dropout

首先，在深度学习里，并不是层次越深，得到的结果越好，层次越深，参数越多，模型越复杂，训练的时间也就越长，还不一定能够得到好的结果。

为什么会出现这样的情况呢？我们知道在使用sigmoid或tanh激活函数时，当输入值很大时，经过激活函数，会得到相对较小的输出，而再经过深层激活，最后得到对应对权重的输出值会越来越小，这会使得，这个权重对于降低损失的作用很少，也就是说，正向传播到后面，已经几乎无作用了，再反向回去，这个权重几乎不会被更新，在靠近损失的地方，相应的权重已经收敛，而前面的权重还几乎是随机状态，这就是梯度消失问题，这种情况就是即使深度很深，得到的结果也不好。

这个时候，我们使用sigmoid和tanh等激活函数，可以使用batchnormalization来进行层级之间的scaling。

对于出现这种情况，我们可以选择其它的激活函数，如ReLU，SeLU，SWISH等。

为什么要选择ReLU？

1. 计算更快
2. 生物可解释性？（神经抑制和兴奋）
3. 使用sigmoid等偏差大
4. 解决梯度消失的问题

使用ReLU激活函数，整个函数是否还可以使用梯度下降法呢？

显然还是可以的，我们知道ReLU是分段函数，前段输出是0，后端输入等于输出。对于未0的部分我们就可以不管，而对于非0部分，即为线性函数，显然可以嵌入到整个函数中的。

因ReLU存在前半部分，会使得部分参数丢失掉，也就不会被更新，所以就产生了很多ReLU的变种，如Leaky ReLU，Pararmetric ReLU，ELU，SeLU等。

* Leaky ReLU: 是对于前半部分给予一个很小的值，默认是0.01，也就说对于那些只有很少影响的参数，我们也使得它可以更新，而不是直接丢弃掉。
* Parametric ReLU: 对于Leaky ReLU只给予固定的参数值，我们觉得不是合理，于是就有了参数版ReLU，这个$\alpha$可以随着训练而得到。
* Maxout: 设定一个参数$k$，$k$为前一层输出成组元素的个数，我们从这个$k$个元素中选出最大的那个元素，传递到后一层。而这个$k$个元素也是学习出来的，从这个理论解释来看，ReLU是Maxout的特殊版本，且maxout按理也是可以使用梯度下降训练的。（$k \ge 2$）Maxout显然形式更多样，可以是多个分段函数的组合。

调整学习率

我们知道固定学习率在深度学习中已经不再适用，在深度学习在localmin收敛时，其梯度应该较小，所以我们应该根据梯度的大小来不断调整合适的学习率，使得函数能够收敛。

除了调整学习率来优化函数外，我们还可以考虑梯度方向的优化，也就是梯度的大小，不仅是与本次的梯度值有关，还与之前的所有累加梯度有关，这个就像RNN中的输出结果不仅跟本次输入有关，还与之前所有状态累加有关。

* 学习率调整的方法有: Adagrad，RMSprop
* 优化梯度的方法有: Momentum
* 学习率优化 + 梯度优化: Adam

Adagrad（ada就是与前面有关的意思，无论是adaboost，还是这边的adagrad）

$$w^{t+1} \gets w^t - \frac{\eta}{\sqrt{\sum_{i=1}^t(g^i)^2}}g^t$$

> 说明：第$t+1$次的更新是由前面$t$次所有梯度和的均方根。因为梯度是不断变化的，这种变化通过这种方式也可以传递给学习率。当前期梯度很大时，其相对学习率还是很大的，而到了后期，梯度越来越小，其学习率也跟着变小。$\sum_{i=0}^t(g^i)^2 = g^0 + g^1 + \cdots + g^t$

RMSprop

在一些复杂的模型中，有些突然走到一个类似localmin中时，我们其实需要一个更大的学习率，来使得其可以跳出这个localmin而进入到下一次的循环，以便可以找到更好的globalmin，这个时候我们使用adagrad就不合适，使用adagrad就会陷入其中而跳不出来。这个时候就可以使用RMSprop，而对于RMSProp，从其公式来看，它是能够控制在本次迭代中是考虑本次的梯度值，还是考虑之前梯度的累加和，这点就很类似LSTM和GRU中的门控机制，用来控制内存中的信息量。

$$w^1 \gets w^0 - \frac{\eta}{\sigma^0}g^0 \hspace{0.5cm} \sigma^0 = g^0$$
$$w^2 \gets w^1 - \frac{\eta}{\sigma^1}g^1 \hspace{0.5cm} \sigma^1 = \sqrt{\alpha(\sigma^0)^2 + (1-\alpha)(g^1)^2}$$
$$w^3 \gets w^2 - \frac{\eta}{\sigma^2}g^2 \hspace{0.5cm} \sigma^2 = \sqrt{\alpha(\sigma^1)^2 + (1-\alpha)(g^2)^2}$$
$$\vdots$$
$$w^{t+1} \gets w^t - \frac{\eta}{\sigma^t}g^t \hspace{0.5cm} \sigma^t = \sqrt{\alpha(\sigma^{t-1})^2 + (1-\alpha)(g^t)^2}$$


梯度优化

Momentum

在现实生活中，我们会有这样的一种情况，当我们在下非常陡的坡时，会因为惯性而冲过头，拿到函数优化中来说，不仅受步伐的影响，同时还会有惯性的力量，这就是Momentum。

在每次决策下降方向时，会同时考虑惯性和梯度，梯度的方向要求往这个方向，而惯性则要求往那个方向，最终决策是惯性和梯度的共同作用结果。

参数的更新与之前的参数和当前的movement都有关，而动能的更新，是在之前动能的基础上减去上次的梯度（其实也可以想象，惯性是随着梯度变化而不断在减小的，所以需要一个惯性的公式，再有一个实际要更新参数的公式，这个参数更新是与之前参数和当前动能相关），具体迭代过程如下：

* 开始初始化参数$\theta^0$，动能$v^0$
* 动能更新$v^1=\lambda v^0 - \eta\nabla L(\theta^0)$，参数更新$\theta^1 = \theta^0 + v^0$
* 下一轮动能更新$v^2=\lambda v^1 - \eta\nabla L(\theta^1)$，参数更新$\theta^2 = \theta^1 + v^2$

初始$v^0=0$，带入上式$v^1=-\eta\nabla L(\theta^0)$，$v^2=-\eta\nabla L(\theta^0) - \eta\nabla L(\theta^1)$，$\cdots$

Adam

adam是RMSProp和Momentum的混合版本，包括使用RMSProp中的参数控制moment和梯度的关系，以及moment中动量更新和参数更新方法。

$$g^t \gets \nabla_{\theta}f_t(\theta_t - 1)$$
$$m^t \gets \beta_1 \cdot m^{t-1} + (1-\beta_1) \cdot (g^t)^2$$
$$v^t \gets \beta_2 \cdot v^{t-1} + (1-\beta_2) \cdot (g^t)^2$$
$$\widehat{m}^t \gets \frac{m^t}{1-\beta_1^t}$$
$$\widehat{v}^t \gets \frac{v^t}{1-\beta_2^t}$$
$$\theta^t \gets \theta^{t-1} - \frac{\alpha\widehat{m}^t}{\sqrt{v^t} + \epsilon}$$

可以看到adam中，对于两个moment，并没有直接使用RMSProp参数更新的结果，而是对这个结果又进行了一次无偏估计，然后再进行参数更新。

早停机制

默认早停机制，是通过对验证集的准确率验证，当其准确率在默认10次验证后，不再提高时，我们可以说就该停止训练了。

正则化

正则化的作用主要是防止训练模型过于复杂，或者是拟合的训练数据太好，而导致在测试集上无法得到好的结果。

从另一个角度来理解，我们不是想将损失降到最低，而是尽量使他接近0。

一般的正则，有L2，L1正则：

$$L'(\theta) = L(\theta) + \lambda\frac{1}{2}\|\theta\|, \text{for} \hspace{0.2cm} \|\theta\|_2=(w_1)^2 + (w_2)^2 + \cdots$$

计算梯度：

$$\frac{\partial L'}{\partial w} = \frac{\partial L}{\partial w} + \lambda w$$

参数更新：

$$w^{t+1} \to w^t - \eta\frac{\partial L'}{\partial w} = w^t - \eta(\frac{\partial L}{\partial w^t} + \lambda w^t) = (1-\eta\lambda)w^t - \eta\frac{\partial L}{\partial w}$$

从上式我们可以看到，L2正则其实就相当于每次对权重参数进行削减，使得权重矩阵中，某些维度的元素接近0。所以说L2正则也被称为权重衰减。

L1正则：

$$L'(\theta) = L(\theta) + \lambda\frac{1}{2}\|\theta\|_1, \text{for}  \hspace{0.2cm}  \|\theta\|_1=|w_1| + |w_2| + \cdots$$

计算梯度：

$$\frac{\partial L'}{\partial w} = \frac{\partial L}{\partial w} + \lambda\text{sgn}(w^t)$$

参数更新：

$$w^{t+1} \to w^t - \eta\frac{\partial L'}{\partial w} = w^t - \eta\bigr(\frac{\partial L}{\partial w^t} + \lambda\text{sgn}(w^t)\bigr) = w^t - \eta\frac{\partial L}{\partial w} - \eta\lambda\text{sgn}(w^t)$$

与L2正则相比，L1正则则每次更新减去一个较小的值，则会使得权重矩阵较均匀。一般来说，对于L1和L2正则的选择，如果函数在远离极值点时，我们会建议使用L2正则，而一般接近极值点，就建议使用L1正则。对于这种问题的讨论，我们在机器学习中有ElasticNet，就是这种L1和L2混合的正则化方式。

对于权重衰减的生物解释，我们的大脑中的神经元间的连接，随着年龄的增大是从少到完整的过程，而一旦进入成熟期，神经元的连接又会慢慢减少，也就是如果某些缔结的链接我们不常用的话，它就会自然而然地消失掉，但是如果后续通过训练取增强它，还是可以再生连接的。

Dropout

Dropout主要是用于训练模型的过拟合问题，一般也是在验证完测试数据后进行的操作。其本质思想是将训练中的数据按照一定的概率移除掉，也就是为了使得模型不要过分依赖所有的训练数据。每次小批量训练时，都是随机按照概率$p$取丢掉一部分神经元。

但在测试集上，我们说对于所有的数据，我们有概率$p$被丢弃，也就是会有概率$1-p$的被保留，对于测试数据就是那部分被保留的数据，我们需要乘以这个概率以保证数据的完整性。

Dropout这种防止过拟合的机制，有点类似模型融合，在随机森林中，每次sampling的样本都是随机的，就如这里的dropout的概率也是随机的。